In [2]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from scipy import stats
import math
import seaborn as sns
%matplotlib inline

In [9]:
initial = pd.read_csv('C:/users/rusty/documents/github/patchseq_gdoc_analysis/patch_seq_log_mouse.csv')

In [10]:
ps = initial[['Date', 'cell type', 'Slice (Lims ID?)', 'Post patch?', 'Post patch pipette R', 
              'SM_QC_PF', 'Bad dates']]
ps['Date'] = pd.to_datetime(ps['Date'], format = '%y%m%d')
ps['Bad dates'] = ps['Bad dates'].astype('category')
ps['cell type'] = ps['cell type'].astype('category')
ps['Post patch?'] = ps['Post patch?'].astype('category')
ps = ps[ps['Bad dates'] != 'x']
ps = ps[ps['SM_QC_PF'] != 'cnt']
ps = ps[ps['cell type'] == 'tdt+']
ps.dropna(subset = ['Post patch?'], inplace =  True)
ps.head()

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-ver

,Date,cell type,Slice (Lims ID?),Post patch?,Post patch pipette R,SM_QC_PF,Bad dates
70,2017-01-09,tdt+,Rbp4-Cre_KL100;Ai14-290392.04.02,Outside-Out,NaN,pass,NaN
71,2017-01-09,tdt+,Rbp4-Cre_KL100;Ai14-290392.04.02,Outside-Out,NaN,fail,NaN
77,2017-01-10,tdt+,Rbp4-Cre_KL100;Ai14-290393.03.02,Outside-Out,NaN,pass,NaN
78,2017-01-10,tdt+,Rbp4-Cre_KL100;Ai14-290393.03.02,Outside-Out,NaN,pass,NaN
79,2017-01-10,tdt+,Rbp4-Cre_KL100;Ai14-290393.04.02,No-Seal,NaN,pass,NaN


In [11]:
ps = ps.drop('Bad dates', 1)
ps = ps.drop('Date', 1)
ps = ps.drop('cell type', 1)
ps = ps.drop('SM_QC_PF', 1)
ps.head()

,Slice (Lims ID?),Post patch?,Post patch pipette R
70,Rbp4-Cre_KL100;Ai14-290392.04.02,Outside-Out,NaN
71,Rbp4-Cre_KL100;Ai14-290392.04.02,Outside-Out,NaN
77,Rbp4-Cre_KL100;Ai14-290393.03.02,Outside-Out,NaN
78,Rbp4-Cre_KL100;Ai14-290393.03.02,Outside-Out,NaN
79,Rbp4-Cre_KL100;Ai14-290393.04.02,No-Seal,NaN


In [ ]:
def nucleated(x):
    nucleus = x[(x["Post patch?"] == "Nucleated") | (((x["Post patch?"] == "nucleus_visible") | (x["Post patch?"] == "nucleus_present")) & (x["Post patch pipette R"] >= 500))]
    return nucleus

def partial_nucleated(y):
    partial = y[(y["Post patch?"] == 'Partial-Nucleus') | (((y['Post patch?'] == 'nucleus_present') | (y['Post patch?'] == 'nucleus_visible')) & (y["Post patch pipette R"] <= 499))]
    return partial

def outside_out(z):
    outside = z[(z["Post patch?"] == 'Outside-Out') | (((z['Post patch?'] == 'nucleus_absent') | (z['Post patch?'] == 'no_nuclues_visible')) & (z['Post patch pipette R'] >= 500))]
    return outside

def no_seal(w): 
    no = w[(w["Post patch?"] == 'No-Seal') | (((w['Post patch?'] == 'nucleus_absent') | (w['Post patch?'] == 'no_nucleus_visible')) & (w['Post patch pipette R'] <= 499))]
    return no
    
def entire_cell(v):
    entire = v[(v["Post patch?"] == 'Entire-Cell') | (v['Post patch?'] == 'entire_cell')]
    return entire